In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas.core.algorithms as algos
import scipy.stats.stats as stats
import re
import traceback
import string
import os


In [2]:
df = pd.read_csv('Cleaned_fights', index_col=[0])
df.head()

,R_fighter,B_fighter,date,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,...,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_stance,B_stance
0,Henry Cejudo,Marlon Moraes,2019-06-08,1.0,True,0,5,0.0,4.0,0.0,...,0.0,0.0,8.0,162.56,162.56,135.0,31.0,32.0,0,0
1,Valentina Shevchenko,Jessica Eye,2019-06-08,1.0,True,11,5,0.0,3.0,0.0,...,2.0,0.0,5.0,165.10,167.64,125.0,32.0,31.0,1,0
2,Tony Ferguson,Donald Cerrone,2019-06-08,1.0,False,6,3,0.0,3.0,0.0,...,6.0,1.0,14.0,180.34,193.04,155.0,36.0,35.0,0,0
3,Jimmie Rivera,Petr Yan,2019-06-08,0.0,False,0,3,0.0,4.0,0.0,...,0.0,0.0,6.0,162.56,172.72,135.0,26.0,29.0,0,2
4,Tai Tuivasa,Blagoy Ivanov,2019-06-08,0.0,False,4,3,0.0,1.0,0.0,...,0.0,0.0,3.0,187.96,190.50,264.0,32.0,26.0,1,1


In [3]:
df2 = pd.read_csv('Cleaned_fights2', index_col=[0])
df2.head()

,R_fighter,B_fighter,date,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_avg_BODY_att,...,R_avg_TOTAL_STR_landed,R_losses,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_stance,B_stance
0,Henry Cejudo,Marlon Moraes,2019-06-08,1.0,True,0,5,0.0,4.0,9.200000,...,69.100000,2.0,8.0,162.56,162.56,135.0,31.0,32.0,0,0
1,Valentina Shevchenko,Jessica Eye,2019-06-08,1.0,True,11,5,0.0,3.0,14.600000,...,102.857143,2.0,5.0,165.10,167.64,125.0,32.0,31.0,1,0
2,Tony Ferguson,Donald Cerrone,2019-06-08,1.0,False,6,3,0.0,3.0,15.354839,...,63.400000,1.0,14.0,180.34,193.04,155.0,36.0,35.0,0,0
3,Jimmie Rivera,Petr Yan,2019-06-08,0.0,False,0,3,0.0,4.0,17.000000,...,50.750000,2.0,6.0,162.56,172.72,135.0,26.0,29.0,0,2
4,Tai Tuivasa,Blagoy Ivanov,2019-06-08,0.0,False,4,3,0.0,1.0,17.000000,...,32.750000,1.0,3.0,187.96,190.50,264.0,32.0,26.0,1,1


In [4]:
df_0 = df[df['Winner']==0]
df_1 = df[df['Winner']==1]

In [5]:
df_02 = df2[df2['Winner']==0]
df_12 = df2[df2['Winner']==1]

In [6]:
print(df_0.shape)
print(df_1.shape)

(1212, 143)
(2284, 143)


In [7]:
print(df_02.shape)
print(df_12.shape)

(1212, 74)
(2284, 74)


In [8]:
df_0 = df_0.sample(n=1000)
df_1 = df_1.sample(n=1000)
df_both = df_0.append(df_1)
df_both.shape

(2000, 143)

In [9]:
df_02 = df_02.sample(n=1000)
df_12 = df_12.sample(n=1000)
df_both2 = df_02.append(df_12)
df_both2.shape

(2000, 74)

In [10]:
df_both = df_both.drop(['R_fighter', 'B_fighter', 'date'], axis=1)
df_both = df_both.astype(float)
df_both.head()

,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,...,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_stance,B_stance
2850,0.0,1.0,5.0,5.0,1.0,0.0,0.0,13.750000,10.000000,17.750000,...,1.0,0.0,8.0,185.42,187.96,185.0,28.0,31.0,1.0,0.0
2107,0.0,0.0,7.0,3.0,1.0,0.0,0.0,1.000000,0.000000,0.000000,...,0.0,0.0,2.0,190.50,190.50,185.0,31.0,31.0,0.0,0.0
1233,0.0,0.0,8.0,5.0,0.0,5.0,0.0,14.428571,10.000000,6.571429,...,0.0,0.0,12.0,175.26,175.26,185.0,32.0,32.0,1.0,0.0
411,0.0,0.0,0.0,3.0,0.0,1.0,0.0,23.714286,14.571429,30.000000,...,1.0,0.0,1.0,172.72,175.26,135.0,37.0,37.0,0.0,0.0
1607,0.0,0.0,6.0,3.0,0.0,3.0,0.0,14.818182,8.090909,9.818182,...,0.0,0.0,9.0,180.34,190.50,155.0,28.0,29.0,0.0,1.0


In [11]:
df_both2 = df_both2.drop(['R_fighter', 'B_fighter', 'date'], axis=1)
df_both2 = df_both2.astype(float)
df_both.head()

,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,...,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_stance,B_stance
2850,0.0,1.0,5.0,5.0,1.0,0.0,0.0,13.750000,10.000000,17.750000,...,1.0,0.0,8.0,185.42,187.96,185.0,28.0,31.0,1.0,0.0
2107,0.0,0.0,7.0,3.0,1.0,0.0,0.0,1.000000,0.000000,0.000000,...,0.0,0.0,2.0,190.50,190.50,185.0,31.0,31.0,0.0,0.0
1233,0.0,0.0,8.0,5.0,0.0,5.0,0.0,14.428571,10.000000,6.571429,...,0.0,0.0,12.0,175.26,175.26,185.0,32.0,32.0,1.0,0.0
411,0.0,0.0,0.0,3.0,0.0,1.0,0.0,23.714286,14.571429,30.000000,...,1.0,0.0,1.0,172.72,175.26,135.0,37.0,37.0,0.0,0.0
1607,0.0,0.0,6.0,3.0,0.0,3.0,0.0,14.818182,8.090909,9.818182,...,0.0,0.0,9.0,180.34,190.50,155.0,28.0,29.0,0.0,1.0


In [12]:
X = df_both.drop(['Winner'], axis=1)
y = df_both['Winner']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=5)

In [13]:
X2 = df_both2.drop(['Winner'], axis=1)
y2 = df_both2['Winner']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=.2, random_state=5)

In [14]:
samples_count = X.shape[0]


train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = X[:train_samples_count]
train_targets = y[:train_samples_count]

validation_inputs = X[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = y[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = X[train_samples_count+validation_samples_count:]
test_targets = y[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

600.0 1600 0.375
200.0 200 1.0
200.0 200 1.0


In [15]:
X_train.shape

(1600, 139)

In [16]:
y_train.shape

(1600,)

In [17]:
X2_train.shape

(1600, 70)

In [22]:
input_size = 139
input_size2 = 70
output_size = 1
hidden_layer_size = 600
    
model = tf.keras.Sequential([
    tf.keras.layers.Dense(139, input_dim=input_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='sigmoid') 
])

custom_optimizer = tf.keras.optimizers.Adam(learning_rate=.0005)

model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 15

max_epochs = 50


early_stopping = tf.keras.callbacks.EarlyStopping(patience=6)


model.fit(X_train, 
          y_train,
          batch_size=batch_size, 
          epochs=max_epochs, 
          #callbacks=[early_stopping],
          #validation_data=(validation_inputs, validation_targets),
          verbose=2
          )  

Epoch 1/50
107/107 - 0s - loss: 1.3028 - accuracy: 0.5188
Epoch 2/50
107/107 - 1s - loss: 0.7002 - accuracy: 0.5387
Epoch 3/50
107/107 - 0s - loss: 0.6833 - accuracy: 0.5713
Epoch 4/50
107/107 - 0s - loss: 0.6821 - accuracy: 0.5606
Epoch 5/50
107/107 - 0s - loss: 0.6758 - accuracy: 0.5719
Epoch 6/50
107/107 - 0s - loss: 0.6744 - accuracy: 0.5694
Epoch 7/50
107/107 - 0s - loss: 0.6645 - accuracy: 0.5956
Epoch 8/50
107/107 - 0s - loss: 0.6654 - accuracy: 0.5894
Epoch 9/50
107/107 - 0s - loss: 0.6559 - accuracy: 0.6069
Epoch 10/50
107/107 - 0s - loss: 0.6663 - accuracy: 0.5987
Epoch 11/50
107/107 - 0s - loss: 0.6577 - accuracy: 0.5975
Epoch 12/50
107/107 - 0s - loss: 0.6521 - accuracy: 0.6150
Epoch 13/50
107/107 - 0s - loss: 0.6511 - accuracy: 0.6275
Epoch 14/50
107/107 - 0s - loss: 0.6424 - accuracy: 0.6244
Epoch 15/50
107/107 - 1s - loss: 0.6632 - accuracy: 0.5994
Epoch 16/50
107/107 - 1s - loss: 0.6501 - accuracy: 0.6156
Epoch 17/50
107/107 - 0s - loss: 0.6429 - accuracy: 0.6156
Epoch 

In [23]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

13/13 [==============================] - 0s 2ms/step - loss: 0.8607 - accuracy: 0.5975


In [24]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.86. Test accuracy: 59.75%


In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 139)               19460     
_________________________________________________________________
dense_10 (Dense)             (None, 600)               84000     
_________________________________________________________________
dense_11 (Dense)             (None, 600)               360600    
_________________________________________________________________
dense_12 (Dense)             (None, 600)               360600    
_________________________________________________________________
dense_13 (Dense)             (None, 600)               360600    
_________________________________________________________________
dense_14 (Dense)             (None, 600)               360600    
_________________________________________________________________
dense_15 (Dense)             (None, 600)              